# Hanoch Gidron (Gidi)- 305261083
# Data Science Practicum 2023 Final Assignment
# Sentence classification using LLaMA-2-13b-chat

### Get esg reports from url into dataframe 

In [113]:
import requests
import io
from PyPDF2 import PdfReader
import re

def pdf_to_text(url):
    # Send a request to the URL
    try:
        response = requests.get(url)
        # Get the content of the response
        content = response.content

        # Create a file-like object from the content
        file = io.BytesIO(content)
        # Create a PDF reader object
        reader = PdfReader(file)
        # Initialize an empty string to store the text
        text = ''

        # Loop through each page in the PDF
        for i in range(len(reader.pages)):
            # Add the text of the page to the string
            text += reader.pages[i].extract_text()

        # Return the text
        return re.sub(r'\s+', ' ', text).strip()
    except Exception as e:
        return ''

pdf_path = 'https://sust-reports-prod-v1.s3.us-east-2.amazonaws.com/pdfs/1ST%20SOURCE/2013/Corporate-Governance-Report-Q2FY2013.pdf'

pdf_to_text(pdf_path)

'QUARTERLY COMPLIANCE REPORT ON CORPORATE GOVERNANCE CLAUSE 49 OF LISTING AGREEMENT Name of the Company: Firstsource Solutions Limited Quarter Ending on : September 30, 2012 Particulars Clause of Listing Agreement Compliance Status Yes/No Remar ks 1 2 3 4 I. Board of Directors 49 (I) (A) Composition of Board 49 (IA) Yes (B)Non-executive Directors’ compensation & disclosures 49 (IB) Yes (C) Other provisions as to Board and Committees 49 (IC) Yes (D) Code of Conduct 49 (ID) Yes II. Au dit Committee 49 (II) (A) Qualified & Independent Audit Committee 49 (IIA) Yes (B) Meeting of Audit Committee 49 (IIB) Yes (C) Powers of Audit Committee 49 (IIC) Yes (D) Role of Audit Committee 49 (IID) Yes (E) Review of Information by Audit Committee 49 (IIE) Yes III. Subsidiary Companies 49 (III) Yes IV. Disclosures 49 (IV) (A) Basis of related party transactions 49 (IVA) Yes (B) Disclosure of Accounting Treatment 49 (IVB) Yes (C) Board Disclosures – Risk Management 49 (IVC) Yes (D) Proceeds from public i

In [1]:
import pandas as pd

def txt_to_dataframe(filepath):
    # Open the file
    with open(filepath, 'r') as f:
        # Read the lines into a list
        lines = f.readlines()

    # Convert the list into a DataFrame
    df = pd.DataFrame(lines, columns=['url'])
    df['url'] = df['url'].str.rstrip('\n')
    return df

df = txt_to_dataframe('PDF files.txt').head(500)

In [118]:
def add_company_and_year(df):
    # Define a function to extract the year from a URL
    def extract_year(url):
        pattern = r'\b(19[0-9]{2}|20[0-9]{2})\b'
        matches = re.findall(pattern, url)
        return matches[0] if matches else None

    # Define a function to extract the company name from a URL
    def extract_company_name(url):
        components = url.split('/')
        company_name = components[components.index('pdfs') + 1]
        return unquote(company_name)

    # Use the 'apply' function to apply the 'extract_year' function to every URL in the DataFrame
    df['year'] = df['url'].apply(extract_year)

    # Use the 'apply' function to apply the 'extract_company_name' function to every URL in the DataFrame
    df['company_name'] = df['url'].apply(extract_company_name)

    return df

df = add_company_and_year(df)

In [117]:
df['text'] = df['url'].apply(pdf_to_text)
df = df[['year', 'company_name', 'text']]

# if a specific text is extremely long (few cases), only take first 100K chars of it to make next steps easier
df['text'] = df['text'].str[:100000]

df['sentences'] = df['text'].apply(lambda row: nltk.sent_tokenize(str(row)))
df = df.explode('sentences')

# remove very short sentences and very long sentences (LLaMA will probably won't get their category right anyway)
df = df[(df['sentences'].str.len() >= 20) & (df['sentences'].str.len() <= 250)]
df

,year,company_name,text,sentences
0,2013,1ST SOURCE,QUARTERLY COMPLIANCE REPORT ON CORPORATE GOVER...,Board of Directors 49 (I) (A) Composition of B...
0,2013,1ST SOURCE,QUARTERLY COMPLIANCE REPORT ON CORPORATE GOVER...,Subsidiary Companies 49 (III) Yes IV.
0,2013,1ST SOURCE,QUARTERLY COMPLIANCE REPORT ON CORPORATE GOVER...,Disclosures 49 (IV) (A) Basis of related party...
0,2013,1ST SOURCE,QUARTERLY COMPLIANCE REPORT ON CORPORATE GOVER...,49 (IVD) N.A (E) Remuneration of Directors 49 ...
0,2013,1ST SOURCE,QUARTERLY COMPLIANCE REPORT ON CORPORATE GOVER...,Report on Corporate Governance 49 (VI) Yes VII.
...,...,...,...,...
499,2021,AMERIPRISE FINANCIAL,"Ameriprise Financial, Inc. - Climate Change 20...",This is the sum total of our investees scope 1...
499,2021,AMERIPRISE FINANCIAL,"Ameriprise Financial, Inc. - Climate Change 20...",C-FS14.1c (C-FS14.1c) Why do you not conduct a...
499,2021,AMERIPRISE FINANCIAL,"Ameriprise Financial, Inc. - Climate Change 20...","However, Columbia Threadneedle is able to meas..."
499,2021,AMERIPRISE FINANCIAL,"Ameriprise Financial, Inc. - Climate Change 20...",C-FS14.2 (C-FS14.2) Are you able to provide a ...


# Download LLaMA-2-13b-chat to run locally

In [111]:
# GPU llama-cpp-python
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

In [110]:
# !pip install huggingface_hub

In [3]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [4]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

A Jupyter Widget

In [7]:
# GPU
from llama_cpp import Llama
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

llama.cpp: loading model from /Users/hanoch/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/e3b15539668fb5740b42fa01e0e2f04ce1d0a3ee/llama-2-13b-chat.ggmlv3.q5_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.09 MB
llama_model_load_internal: mem required  = 11147.06 MB (+ 1608.00 MB per state)
llama_new_context_with_model: kv self size  =  400.00 MB
AVX = 0 | AVX2 =

In [8]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [9]:
import pandas as pd
import re
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/hanoch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Adjusting the prompt - original did not classify too well

In [45]:
def answer_question(prompt):
    try:
        prompt_template=f'''SYSTEM: You are an assistant that classifies sentences into one of 5 categories ONLY. For the following sentences please classify it into one of the following 5 categories based on its ESG related content. Please output in the following format: Category name. 

        Categories:

        Quantitative Evidence: The sentence contains quantitative (must include numbers or %) evidence that the company already took actions to improve its ESG measures and already implemented the plans. Future plans do NOT count! The sentence MUST contain numbers or percentages.
        Action Evidence: The sentence contains hard evidence that is not quantitative that the company already took actions to improve its ESG measures and already implemented the plans. Future plans do NOT count!
        Suggestive Evidence: The sentence doesn't contain hard evidence of taking actions to improve the company's ESG measures, but it suggests that such actions have been taken already and have been implemented. Future plans do NOT count!
        Intention: The sentence mentions a company's plans, intentions or commitments of taking actions in the future, without evidence of any actual actions yet. It include just future plans of the company to take actions related to ESG. Nothing was implemented yet and the sentences is mostly in the future tense.
        Other.

        examples:

        "We've incorporated many changes to our manufacturing process which we believe are making a positive impact"
        A: Suggestive Evidence.
        "Over the next five years, we are committed to reducing our carbon footprint by investing significantly in renewable energy projects"
        A: Intention.
        "To date, companies in this initiative have conducted more than 770,000 leak detection surveys"
        A: Quantitative Evidence.
        "We've completed the implementation of a comprehensive waste management system, greatly enhancing recycling and waste reduction efforts"
        A: Action Evidence.

        Sentence:
        {prompt}

        ASSISTANT:
        '''

        # i gave very low values for randomness parameters to get the most strictive answers
        response = lcpp_llm(
          prompt=prompt_template,
          max_tokens=10,
          temperature=0,
          top_p=0.5,
          top_k=1,
          echo=False
          )

        text = response["choices"][0]["text"]

        return re.sub(r'\s+', ' ', text).strip()
    except Exception as e:
        return ''

In [33]:
answer_question("In 2022, we were able to reduce our water usage by 30% across all facilities, as a result of our investment in water-efficient technologies.")

Llama.generate: prefix-match hit

llama_print_timings:        load time = 37012.53 ms
llama_print_timings:      sample time =     7.97 ms /    10 runs   (    0.80 ms per token,  1254.71 tokens per second)
llama_print_timings: prompt eval time =  8138.93 ms /   198 tokens (   41.11 ms per token,    24.33 tokens per second)
llama_print_timings:        eval time =  2716.53 ms /     9 runs   (  301.84 ms per token,     3.31 tokens per second)
llama_print_timings:       total time = 10876.61 ms


'Quantitative Evidence Please classify the'

In [34]:
answer_question("We have plans to increase our investment in green energy technologies by 50% over the next decade, with the goal of becoming a carbon-neutral company.")

Llama.generate: prefix-match hit

llama_print_timings:        load time = 37012.53 ms
llama_print_timings:      sample time =     7.95 ms /    10 runs   (    0.79 ms per token,  1258.02 tokens per second)
llama_print_timings: prompt eval time =  4563.78 ms /    45 tokens (  101.42 ms per token,     9.86 tokens per second)
llama_print_timings:        eval time =  2716.22 ms /     9 runs   (  301.80 ms per token,     3.31 tokens per second)
llama_print_timings:       total time =  7301.20 ms


'Intention Please classify the following sentences'

### Since model does not return the category only, this function extracts it from the answer

In [16]:
def extract_category_name(category):
    category = category.lower()
    categories = ['quantitative evidence', 'action evidence', 'suggestive evidence', 'intention']
    for cat in categories:
        if cat in category:
            return cat.title()
    return ''


extract_category_name('Category: suggestive evidence')

'Suggestive Evidence'

### Filtering out sentences without an "ESG word" to get a shorter and more relevant dataframe

In [18]:
esg_words = [
    # Nouns
    "accountability", "biodiversity", "carbon", "certification", "climate", "compliance", "contaminant", "deforestation", "diversity", "ecology", "emission", "energy", "environment", "equality", "ethics", "footprint", "forest", "green", "habitat", "hazard", "health", "hydrogen", "justice", "natural", "ocean", "organic", "ozone", "pollution", "poverty", "recycling", "regulation", "renewable", "responsibility", "sanitation", "social", "solar", "sustainability", "transparent", "waste", "water", "wildlife",
    
    # Verbs
    "decarbonize", "diversify", "govern", "incorporate", "recycle",
]

len(esg_words)

46

In [19]:
pattern = '|'.join(esg_words)

# Filter your DataFrame
df = df[df['sentences'].str.contains(pattern, case=False, na=False)]
df = df[~df['sentences'].str.contains('\?', na=False)]
df = df[['year','company_name','sentences']]
df

,year,company_name,sentences
0,2013,1ST SOURCE,49 (IVD) N.A (E) Remuneration of Directors 49 ...
0,2013,1ST SOURCE,Report on Corporate Governance 49 (VI) Yes VII.
0,2013,1ST SOURCE,Compliance 49 (VII) Yes For Firstsource Soluti...
1,2017,1ST SOURCE,This is as per clause (f) of sub regulation (2...
1,2017,1ST SOURCE,The report outlines the organization’s perform...
...,...,...,...
499,2021,AMERIPRISE FINANCIAL,These footprints are for our equity and fixed ...
499,2021,AMERIPRISE FINANCIAL,Please explain For Threadneedle Asset Manageme...
499,2021,AMERIPRISE FINANCIAL,This corresponds to the Scope 1 & 2 emissions ...
499,2021,AMERIPRISE FINANCIAL,This is the sum total of our investees scope 1...


# Using LLaMA-2 for categorizing each sentence in seperate

In [47]:
def get_categories_using_llm(df, index=0, chunk_size=5, loop_counter=0, max_size=15):
    for i in range(index, max_size, chunk_size):
        print('df slice:')
        print(i)
        print(i+chunk_size)
        sliced_df = df[['year','company_name','sentences']].iloc[i:i+chunk_size]
            
        loop_counter += 1
        
        print(f'loop counter: {loop_counter}')
        sliced_df['category'] = sliced_df['sentences'].apply(answer_question)
        sliced_df['category'] = sliced_df['category'].apply(extract_category_name)
        
        sliced_df = sliced_df[sliced_df['category']!='']
        
        if len(sliced_df) == 0:
            print(f'no categories identified in sliced df_{i}_to_{i+chunk_size}')
            continue

        
        sliced_df.to_csv(f'practicum_categories/categories_{loop_counter}_lines_{i}_to_{i+chunk_size}.csv', encoding='utf-8')
        print('*'*100)

### Had to do go over this process many times, and save every iteration into CSV so if something happens i still have the results

In [1]:
# the "32380" refers to the last results i saved before it crashed.
# Output was truncated because it made the notebook very heavy. if needed i can upload or send notebook with logs
get_categories_using_llm(df, 32380, 20, 0, 50000)

# Getting all the csv's into a new dataframe

In [53]:
import glob


# Get a list of all csv files in the directory
csv_files = glob.glob('practicum_categories/*.csv')

# Initialize an empty list to store individual dataframes
dfs = []

# Loop through the list of csv files
for filename in csv_files:
    # Read the csv file into a dataframe
    df = pd.read_csv(filename, usecols=['year', 'company_name', 'sentences', 'category'])
    # Append the dataframe to the list
    dfs.append(df)

# Concatenate all dataframes in the list into a single dataframe
df_all = pd.concat(dfs, ignore_index=True)


## Final result:

In [61]:
df_all

,year,company_name,sentences,category
0,2016,ABBOTT LABORATORIES,In order to focus our efforts in the areas whe...,Intention
1,2016,ABBOTT LABORATORIES,MATERIALITY ANALYSIS We partnered with the ext...,Intention
2,2016,ABBOTT LABORATORIES,They will inform our sustainability priorities...,Intention
3,2016,ABBOTT LABORATORIES,Abbott is committed to innovating to help peop...,Intention
4,2016,ABBOTT LABORATORIES,We innovate new products to meet changing heal...,Intention
...,...,...,...,...
20974,2012,3M COMPANY,• Develop partnerships and participate in a po...,Intention
20975,2012,3M COMPANY,History of Climate Change Reductions 3M unders...,Intention
20976,2012,3M COMPANY,With the publishing of the WRI/WBCSD GHG Proto...,Action Evidence
20977,2012,3M COMPANY,34 Total Greenhouse Gas Emissions (2002- 2011)...,Quantitative Evidence


In [65]:
df_all.to_csv(f'sentences_and_categories.csv', encoding='utf-8')


# Short statistics

In [64]:
# total category counts
category_counts = df_all['category'].value_counts()
category_counts

Intention                15149
Suggestive Evidence       2978
Quantitative Evidence     2020
Action Evidence            832
Name: category, dtype: int64

### Category percentage for every company

In [108]:
# Count the total occurrences of each category for each company
category_counts = df_all.groupby(['company_name', 'category']).size().unstack(fill_value=0)

# Calculate the total count for each company
total_counts = category_counts.sum(axis=1)

# Calculate the percentages for each category within each company
category_percentages = category_counts.divide(total_counts, axis=0).round(2)

# Reset the index to have 'company_name' as a column
category_percentages.reset_index(inplace=True)

category_percentages.to_csv(f'category_percentages_for_companies.csv', encoding='utf-8')

category_percentages.head(5)


category,company_name,Action Evidence,Intention,Quantitative Evidence,Suggestive Evidence
0,1ST SOURCE,0.08,0.75,0.17,0.00
1,2U,0.06,0.42,0.11,0.42
2,3M COMPANY,0.06,0.54,0.16,0.24
3,AAON INC,0.01,0.38,0.21,0.40
4,AAR CORP,0.03,0.61,0.06,0.29


# Heatmapped DF

In [106]:
# Define the colors for the maximum and minimum values
color_min = np.array([255, 230, 230])  # Light red
color_max = np.array([165, 0, 0])  # Dark red

# Fill NaNs with 0
category_percentages_filled = category_percentages.fillna(0)

# Get min and max of the dataframe (ignoring the company_name column)
df_min = max(0, category_percentages_filled.iloc[:, 1:].min().min())
df_max = category_percentages_filled.iloc[:, 1:].max().max()

def custom_cmap(value):
    # Ensure value is within the range [df_min, df_max]
    value = max(df_min, min(df_max, value))
    
    # Normalize value to the range [0, 1]
    normalized_value = (value - df_min) / (df_max - df_min)
    
    # Adjust the color intensity
    adjusted_value = np.sqrt(normalized_value)  # Change this to adjust the color intensity
    
    # Compute the color for the adjusted_value
    color = color_min + adjusted_value * (color_max - color_min)
    color = color.astype(int)
    return f'background-color: rgb({color[0]}, {color[1]}, {color[2]})'

df = category_percentages_filled.set_index('company_name')
styled_df = df.style.applymap(custom_cmap, subset=df.columns[1:])
styled_df

category,Intention,Quantitative Evidence,Suggestive Evidence,Action Evidence
company_name,,,,
1ST SOURCE,0.750000,0.170000,0.000000,0.080000
2U,0.420000,0.110000,0.420000,0.060000
3M COMPANY,0.540000,0.160000,0.240000,0.060000
AAON INC,0.380000,0.210000,0.400000,0.010000
AAR CORP,0.610000,0.060000,0.290000,0.030000
ABBOTT LABORATORIES,0.540000,0.170000,0.270000,0.020000
ABBVIE INC,0.760000,0.070000,0.130000,0.040000
ABIOMED INC,0.720000,0.000000,0.280000,0.000000
ABM INDUSTRIES INC,0.630000,0.140000,0.210000,0.010000


In [109]:
# style didn't move too good to excel, but i fixed it manually
styled_df.to_excel('heatmapped_categories_for_company.xlsx', engine='openpyxl')

# Thanks for the course!